# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [20]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [21]:
# Ваше решение
def func(x):
    if 'не более' in x.Норматив:
        return eval(str(x['Результат анализа']) + re.sub('не более ', '<=', x.Норматив))
    else:
        interval = re.sub('в пределах ', '', x.Норматив).split('-')
        return eval(interval[0] + '<=' + str(x['Результат анализа']) + ' and '\
                    + str(x['Результат анализа']) + '<=' + interval[1])
df['Результат анализа'][2] = 0
df.Норматив[1] = 'не более 3'
df.Норматив = df.Норматив.str.replace(',','.')
df['Вывод'] = df.apply(func, axis=1, result_type='reduce')

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [32]:
# Ваше решение
import random
n = [10, 100, 1000, 10000, 100000, 1000000]
for i in n:
    success_counter = 0
    for j in range(i):
        box = [1, 1, 1, 1, 1, 0, 0, 0, 0]
        for _ in range(3):
            random.shuffle(box)
            del box[0]
        if sum(box) == 2:
            success_counter+=1
    print(f'Вероятность успеха при {i} попытках - {success_counter/i}')


Вероятность успеха при 10 попытках - 0.1
Вероятность успеха при 100 попытках - 0.13
Вероятность успеха при 1000 попытках - 0.12
Вероятность успеха при 10000 попытках - 0.1177
Вероятность успеха при 100000 попытках - 0.11923
Вероятность успеха при 1000000 попытках - 0.118992


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [38]:
# Ваше решение
n = [10, 100, 1000, 10000, 100000, 1000000]
for i in n:
    success_counter = 0
    for j in range(i):
        box = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
        case = [1, 0]
        for _ in range(len(case)):
            random.shuffle(box)
            if box[0] == case[0]:
               del case[0]
            del box[0]
        if len(case) == 0:
            success_counter+=1

    print(f'Вероятность успеха при {i} попытках - {success_counter/i}')

Вероятность успеха при 10 попытках - 0.4
Вероятность успеха при 100 попытках - 0.28
Вероятность успеха при 1000 попытках - 0.212
Вероятность успеха при 10000 попытках - 0.2339
Вероятность успеха при 100000 попытках - 0.23293
Вероятность успеха при 1000000 попытках - 0.233506


In [34]:
len([1, 1, 1, -1, -1, -1, -1, -1, -1, -1])

10